# 1.6 - 1.7. Mô tả và đánh giá dữ liệu & Tiền xử lí dữ liệu

* Load toàn bộ review vào một dataframe duy nhất.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.utils as Utils
import modules.processor as Processor
import numpy as np
import pandas as pd
import enchant
import random

from sklearn.utils import shuffle

In [3]:
# Lấy tất cả các directory path của các lần ta tiến hành crawl data
dir_paths = Utils.getAllFolderPath("./data/product_reviews/")

dir_paths

['./data/product_reviews/product_reviews_02/',
 './data/product_reviews/product_reviews_03/',
 './data/product_reviews/product_reviews_01/',
 './data/product_reviews/product_reviews_00/']

In [4]:
# Đọc toàn bộ các review từ các file csv
reviews = Utils.readReviews(dir_paths)

In [5]:
reviews.head()

,raw_comment,rating
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,5
1,đẹp lắm ạ,5
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,5
3,đẹp lắm ạ,5
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",5


In [6]:
print("Tập dữ liệu có {} bình luận.".format(reviews.shape[0]))

Tập dữ liệu có 278159 bình luận.


* Đếm tần số xuất hiện của từng rating.

In [7]:
reviews['rating'].value_counts()

5    260555
4      9646
3      4194
1      2308
2      1456
Name: rating, dtype: int64

> **Nhận xét**:
> * Nhìn chung tuy ta crawl được hơn 200,000 quan sát nhưng có sự chênh lệch lớn giữa các rating.
> * Nhìn qua ta thấy đa phần là các rating được đánh giá 5 sao, điều này cũng dễ hiểu vì hệ thống recommend của Shopee sẽ ưu tiên gợi ý cho khách hàng những sản phẩm có đánh giá tốt. Và sẽ hạn chế hoặc thậm chí là ko gợi ý các mặc hàng bị đánh giá kém. Nên với địa vị là người đi trộm dữ liệu như chúng ta thì ko có cách nào khắc phục điều này.
> * Bây giờ, do ta cần chi ra hai lớp là negative và positive nên những comment mà `rating` $\geq 4$ sẽ được cho vào nhóm positive, ngược lại là nhóm negative. 

* Tiến hành label cho `reviews` với các giá `rating` $< 4$ sẽ thuộc nhóm negative còn lại là nhóm positive.

In [8]:
reviews = Utils.labelRating(reviews)

In [9]:
reviews.head()

,raw_comment,rating,label
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,5,1
1,đẹp lắm ạ,5,1
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,5,1
3,đẹp lắm ạ,5,1
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",5,1


* Bây giờ ta sẽ xóa đi feature `rating` vì về sau ta sẽ ko cần dúng đến nó nữa, và tiến hành đếm số lượng quan sát của từng nhóm trên `label`

In [10]:
reviews = reviews.drop(columns=['rating'])

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (278159, 2)
1    270201
0      7958
Name: label, dtype: int64


,raw_comment,label
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,1
1,đẹp lắm ạ,1
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,1
3,đẹp lắm ạ,1
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1


<hr>

* Một trong những vấn đề đầu tiên và tối quan trong khi xử lí với dữ liệu văn bản là kiểm tra xem liệu text `a` có cùng cách biểu diễn với text `b` hay không.
* Một ví dụ dễ hiểu là giả sử ta có biến `a = 'đẹp'` và biến `b = 'đep'`, nhưng khi ta compare hai biến này `a == b` thì kết quả sẽ ra False, nguyên nhân là do chúng sử dụng mã hóa unicode khác nhau, có thể `a` dùng unicode-8 và `b` dùng unicode-16.
* Vậy điều đầu tiên ta cần làm là phải đưa tất cả các text về cùng một **chuẩn** duy nhất, ta có thể làm điều này bằng cách sử dụng `unicodedata.normalize()` từ package chuẩn `unicodedata` của Python. _(tham khảo thêm tại đây [https://www.kite.com/python/docs/unicodedata.normalize](https://www.kite.com/python/docs/unicodedata.normalize))_
* Dưới đây là ví dụ cho trường hợp này:

In [11]:
import unicodedata

a = 'đẹp, rất hài lòng'
b = 'đẹp, rất hài lòng'

a == b

False

In [12]:
''' Sử dụng chuẩn NFD '''
a = unicodedata.normalize('NFD', a)
b = unicodedata.normalize('NFD', b)

print("String a: {}, Type a: {}".format(a, type(a)))
print("String b: {}, Type b: {}".format(b, type(b)))

String a: đẹp, rất hài lòng, Type a: <class 'str'>
String b: đẹp, rất hài lòng, Type b: <class 'str'>


In [13]:
a == b

True

* Bây giờ, ta sẽ tạo một feature có tên là `normalize_comment`, trải qua 2 bước:
  * `lower()` cho text.
  * Chuẩn hóa bằng `unicodedata.normalize()`

In [14]:
reviews['normalize_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt))

reviews.head()

,raw_comment,label,normalize_comment
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
1,đẹp lắm ạ,1,đẹp lắm ạ
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdfas...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
3,đẹp lắm ạ,1,đẹp lắm ạ
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau"


* Ta cũng sẽ chuẩn hóa cho cột `raw_comment` nhưng không `lower()` chúng.

In [15]:
reviews['raw_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt, False))

reviews.head()

,raw_comment,label,normalize_comment
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
1,đẹp lắm ạ,1,đẹp lắm ạ
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
3,đẹp lắm ạ,1,đẹp lắm ạ
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau"


<hr>

* Chúng ta biết rằng, các comment của các sản phẩm đôi khi sẽ chứa các URL do người bán hàng chèn vào để giúp khách hàng có thể click vào để xem các mặt hàng khác, chúng là các noise sample mà ta cần phải loại bỏ khỏi dataset của chúng ta.
* Hình dưới đây là kết quả cho ra khi ta thử search cụm từ `http` thì nó cho ra hơn 600 mẫu dữ liệu chứa URL. Ta cần loại bỏ các mẫu này.<br>
  ![](./images/04.png)

In [16]:
reviews['contain_url'] = reviews['normalize_comment'].apply(lambda cmt: Processor.containsURL(cmt))

Processor.printAfterProcess(reviews, 'contain_url')
reviews.head()

Shape: (278159, 4)
0    277589
1       570
Name: contain_url, dtype: int64


,raw_comment,label,normalize_comment,contain_url
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,0
1,đẹp lắm ạ,1,đẹp lắm ạ,0
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,0
3,đẹp lắm ạ,1,đẹp lắm ạ,0
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",0


* Bây giờ chúng ta chỉ sẽ lấy các comment mà không chứa URL

In [17]:
reviews = reviews[reviews['contain_url'] == 0]
reviews = reviews.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột `contain_url`

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (277589, 3)
1    269633
0      7956
Name: label, dtype: int64


,raw_comment,label,normalize_comment
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
1,đẹp lắm ạ,1,đẹp lắm ạ
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
3,đẹp lắm ạ,1,đẹp lắm ạ
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau"


> **Nhận xét**
> * Đa phần là các bình luận thuộc nhóm positive sẽ chứa các URL, cũng dễ hiểu vì họ quảng cáo mà 😅.

<hr>

* Tiếp theo, ta cũng cần xóa các comment mà chữ IN HOA chiếm quá 50% độ dài comment, các comment này khả năng cao cũng là quảng cáo, vì người bán họ muốn làm nổi bật bình luận này lên so với các bình luận còn lại.

In [18]:
reviews['contain_adv'] = reviews['raw_comment'].apply(lambda cmt: Processor.containAdvertisement(cmt))

Processor.printAfterProcess(reviews, 'contain_adv')
reviews.head()

Shape: (277589, 4)
False    270936
True       6653
Name: contain_adv, dtype: int64


,raw_comment,label,normalize_comment,contain_adv
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,False
1,đẹp lắm ạ,1,đẹp lắm ạ,False
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,False
3,đẹp lắm ạ,1,đẹp lắm ạ,False
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",False


> **Nhận xét**
> * Các mẫu tìm năng chứa quảng cáo khá cao, lên đến hơn 3000 sample. Ta có thể xóa chúng.

In [19]:
reviews = reviews[reviews['contain_adv'] == 0]
reviews = reviews.drop(columns=['contain_adv']).reset_index(drop=True) # xóa cột `contain_adv`

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (270936, 3)
1    263084
0      7852
Name: label, dtype: int64


,raw_comment,label,normalize_comment
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
1,đẹp lắm ạ,1,đẹp lắm ạ
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...
3,đẹp lắm ạ,1,đẹp lắm ạ
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau"


> **Nhận xét**
> * Lại một lần nữa các comment có khả năng cao là quảng cáo này lại đa phần là thuộc nhóm positive.

<hr>

* Nhìn qua các comment, ta sẽ thấy có các comment chứa emoji như hình dưới đây:<br>
  ![](./images/05.png)

* Đây là "vốn quý" góp phần làm tăng sức mạnh cho model, nếu ta thực hiện bước loại bỏ các kí tự đặc biệt trước khi ta tách các emoji ra, thì ta đã vô tình xóa luôn các emoji này, vì các emoji thực chất được xây dựng dựa trên các kí tự đặc biệt.
* Như hình trên, rõ ràng ta thấy được emoji góp phần ta hiểu được một comment là positive hay negative.
* Ta sẽ sử dụng một gói của python là `emojis`:
  ```shell
  pip3 install emojis
  ```
  gói này sẽ giúp ta tách các emoji ra khỏi bình luận.
* Ta sẽ chứa toàn bộ emoji của một comment qua cột tương ứng là `emoji`

In [20]:
reviews['emoji'] = reviews['raw_comment'].apply(lambda cmt: Processor.extractEmoji(cmt))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
1,đẹp lắm ạ,1,đẹp lắm ạ,
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
3,đẹp lắm ạ,1,đẹp lắm ạ,
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,"áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",


In [21]:
",".join(reviews['emoji'])[:200]

',,,,,,,,,,😻,,,,👍,,,,,,,✨,,,,,,,,,,,,,😜,,,,,,,,,❤️,,🤣,,,👍,,,,😘,,😘,,,,,,,,,,,,,❤️ 👍,,❤️ 👍,,,❤️,\U0001f971,🎉,,❤️,\U0001f971,🎉,,,,,,,,,❤️,,,😅,,,🖤 💜 \U0001f970,,,🖤 💜 \U0001f970,,🙆,,🙆,,,,,,,,,,,,\U0001f97a,,,,,,,,,,,,,,,😀 😊 \U0001f92a 😅 ❤️,,,,,♂️ 🤦 🙏,,,,,,,,,,'

<hr>

* Tiếp theo, ta sẽ loại bỏ dấu câu, kí tự đặc biệt

In [22]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt.lower()))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
1,đẹp lắm ạ,1,đẹp lắm ạ,
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
3,đẹp lắm ạ,1,đẹp lắm ạ,
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,áo đẹp giá rẻ sẽ ủng hộ vào lần sau,


<hr>

* Tiếp theo, ta cần chuẩn lại các từ bị dupplicate như: _chờiiiiiii ơiiiiiii, xinhhhhhhh quá, đẹp xỉuuuuuuuuuuuu_ thành _chời ơi, xinh quá, đẹp xỉu_.
* Tuy nhiên có một vấn đề xảy ra, giả sử trong comment có các từ tiếng anh như "_feedback_", thì nó sẽ thành "_fedback_", nên ta sẽ thực hiện bước này ở phần sau:

```python
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeDuplicateLetters(cmt))

```

<hr>

* Tiếp theo, chúng ta sẽ chuẩn lại một vài từ viết tắt cơ bản.
* File `modules/dependencies/abbreviate.txt` chứa các từ viết tắt cơ bản mà giới trẻ hay dùng comment, ta có thể bổ sung theo thời gian.

In [23]:
# xây dựng dictionary cho các từ viết tắt
abbreviate = Utils.buildDictionaryFromFile("./modules/dependencies/abbreviate.txt")

# test
abbreviate['okela']

'ok'

In [24]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.replaceWithDictionary(cmt, abbreviate))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
1,đẹp lắm ạ,1,đẹp lắm ạ,
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,
3,đẹp lắm ạ,1,đẹp lắm ạ,
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,áo đẹp giá rẻ sẽ ủng hộ vào lần sau,


<hr>

* Bây giờ ta sẽ tiến hành xóa các từ vô nghĩa trong comment, ví dụ như hình dưới đây:<br>
  ![](./images/06.png)
  những bình luận này mặc dù thuộc lớp positive nhưng nó là các noise sample, có thể các comment này dùng để comment cho có để nhận shopee xu khi đánh giá sản phẩm.
* Kế tiếp, ta nên xóa các sample mà khả năng cao ko là tiếng việt, vì sao ta làm bước này, đơn giản thôi đây là shopee việt nam, và các comment cố ý bằng tiếng anh, tiếng hàn, tiếng trung của các "thánh làm màu" sẽ là các noise sample khiến model ta bị giảm hiệu năng.
* Nhưng làm sao ta có thể thực hiện điều này, cách đơn giản nhất là ta có thể sử dụng các package như `textblob`, `googletrans`,... các package này chứa các function giúp ta detect language cho text, tuy nhiên hạn chế là chúng chỉ cho tối đa khoảng 200 request một ngày thôi, và số mẫu của chúng ta hiện tại là quá lớn. Ở đây ta có file `modules/dependencies/vocabulary.txt` chứa hơn 17000 từ đơn phổ biến của tiếng việt.
* Vậy cách đơn giản hơn là ta có thể xây dựng một dictionary chứa các từ đơn của tiếng việt, với mỗi comment, nếu số lượng từ ko tìm thấy trong dictionary này lớn hơn số từ được tìm thấy trong dictionary thì khả năng cao đây là một comment làm màu.
* Tuy nhiên, vẫn có một vài từ tiếng anh mà ta cần giữ lại như shipper, ta sẽ sử dụng package `enchant` để check một từ có phải là từ tiếng anh hay không.
  ```shell
  pip3 install pyenchant
  ```
* Ở các bước phía trên, ta đã đề cập đến việc xóa các từ bị dupplicate kí tự, ta sẽ thực hiện nó ở trong bước này.

In [25]:
# hơn 17 ngàn từ đơn trong tiêng việt
vocabularies = Utils.buildDictionaryFromFile('./modules/dependencies/vocabulary.txt', True)
english_voca = enchant.Dict('en_US') # english if a word is english

In [26]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeNoiseWord(cmt, vocabularies, english_voca))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,đẹp,
1,đẹp lắm ạ,1,đẹp lắm ạ,
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,đẹp,
3,đẹp lắm ạ,1,đẹp lắm ạ,
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,áo đẹp giá rẻ sẽ ủng hộ vào lần sau,


<hr>

* Tiếp theo ta sẽ remove stopword, chúng ta sẽ sử dụng stopword trong file `modules/dependencies/stopwords.txt`. Ta không nên sử dụng các stopword được build sẵn trên mạng nhất là cho tiếng việt, vì chưa chắc các từ này đã hợp với dữ liệu hiện tại của chúng ta.
* Ví dụ nhiều stopword set loại bỏ từ "**nhưng**", tuy nhiên từ này khả năng cao là quan trọng, giả sử ta có câu này:
  * _shop giao hàng chậm **nhưng** giao đúng hàng, ủng hộ shop_, thì nhờ từ **nhưng** này mà model ta có khả năng phân biệt được nó là positive hay negative.
* Ngoài ra với một file txt như vậy, ta có thể bổ sung stopword sau này.

In [27]:
stopwords = Utils.buildListFromFile("./modules/dependencies/stopwords.txt")

In [28]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeStopwords(cmt, stopwords))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,đẹp,
1,đẹp lắm ạ,1,đẹp lắm,
2,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,đẹp,
3,đẹp lắm ạ,1,đẹp lắm,
4,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,áo đẹp giá rẻ sẽ ủng hộ vào lần sau,


<hr>

* Tiếp theo, ta loại bỏ các empty và duplicate `normalize_comment`.

In [29]:
reviews = Processor.removeEmptyOrDuplicateComment(reviews)

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (217218, 4)
1    210117
0      7101
Name: label, dtype: int64


,raw_comment,label,normalize_comment,emoji
0,asdfasdf đẹp ádfasdfasdfasdfasdfasdfasdfasdf...,1,đẹp,
1,đẹp lắm ạ,1,đẹp lắm,
2,"Áo đẹp, giá rẻ, sẽ ủng hộ vào lần sau",1,áo đẹp giá rẻ sẽ ủng hộ vào lần sau,
3,Chất OK đẹp 2 bộ lớn \nBộ bé thì kg...,1,chất ok đẹp bộ lớn bộ bé kg mê lắ...,
4,Các bạn đặt tăng lên 1 size cho thoải m...,1,các bạn đặt tăng lên size cho thoải má...,


<hr>

* Train test split, ta thấy rằng giữa hai nhóm positive và negative có chênh lệnh lớn, nên tập train data của ta sẽ bằng $0.8 * \min(\mathrm{size}(positive), \mathrm{size}(negative)) * 2$.

In [30]:
half_min_size = min(reviews['label'].value_counts())

half_min_size

7101

In [31]:
reviews_positive = reviews[reviews['label'] == 1]
reviews_negative = reviews[reviews['label'] == 0]

reviews_positive = shuffle(reviews_positive)
reviews_positive = reviews_positive.reset_index(drop=True)

In [32]:
positive_index = random.sample(range(0, reviews_positive.shape[0]), half_min_size)

positive_index[:10]

[152523, 205225, 115014, 23971, 195492, 191617, 134941, 148560, 102373, 3504]

In [33]:
reviews_positive2 = reviews_positive.iloc[positive_index,:]

reviews_positive2.head()

,raw_comment,label,normalize_comment,emoji
152523,"Giao hàng nhanh, mua hqua hnay đã có rồi....",1,giao hàng nhanh mua có săn sale không nên...,
205225,Giá cả hợp lý. Đường may có vẻ chắ...,1,giá cả hợp lý đường may có vẻ chắc...,🍀
115014,Cũng tạm được mà mỏng quá so với gia...,1,cũng tạm được mà mỏng quá so giá gầ...,🤦‍♀️
23971,Hàng chất lượng với giá tiền,1,hàng chất lượng giá tiền,
195492,"Áo phom chuẩn size, vải nếu mầu trắn...",1,áo phom chuẩn size vải nếu mầu trắng...,


* Đây là tập data mà hai nhóm positive và negative cân bằng nhau

In [34]:
normalize_reviews = pd.concat([reviews_negative, reviews_positive2], axis=0)
normalize_reviews = normalize_reviews.reset_index(drop=True)

Processor.printAfterProcess(normalize_reviews)
normalize_reviews.head()

Shape: (14202, 4)
1    7101
0    7101
Name: label, dtype: int64


,raw_comment,label,normalize_comment,emoji
0,Quần giặt xong co ngắn mất 5 phân. Ne...,0,quần giặt xong co ngắn mất phân nên ...,
1,Vải áo mặc nóng \nCòn vải quần dễ ...,0,vải áo mặc nóng còn vải quần dễ xù...,❤️
2,"Đóng gói cẩn thận, giao đủ số lượn...",0,đóng gói cẩn thận giao đủ số lượng...,
3,Quần quá to mọi người ạ còn dài nx ...,0,quần quá to mọi người còn dài mặc d...,
4,Vải thừa hơi nhiều\nGiao hàng nhanh .!\...,0,vải thừa hơi nhiều giao hàng nhanh thủ...,


* Ghi ra file

In [35]:
normalize_reviews.to_csv("./data/normalize_reviews.csv", index=False)

* Bây giờ ta sẽ ghi phần bù còn lại của `review_positive` vào file, ta có thể dùng nó cho việc evaluate model sau này.

In [36]:
reviews_positive3 = reviews_positive[~reviews_positive.index.isin(positive_index)]
reviews_positive3 = reviews_positive3.reset_index(drop=True)

reviews_positive3.head()

,raw_comment,label,normalize_comment,emoji
0,Bộ rộng thoảng mái mặc mát,1,bộ rộng thoảng mái mặc mát,
1,Sản phẩm xài tốt,1,sản phẩm xài tốt,
2,Xinh ạ. Nhìn sáng. Hàng về y hìnhhh. Si...,1,xinh nhìn sáng hàng về y hình siêu xinh,
3,Nhận đc hàng rồi giao nhanhqqdvbnxghbcxeg...,1,nhận được hàng giao,
4,Ví đẹp. Có hộp kèm theo. Shop gửi sai ...,1,ví đẹp có hộp kèm theo gửi sai màu tu...,


In [37]:
reviews_positive3.to_csv("./data/complement_positive_reviews.csv", index=False)

# 1.8. Tài liệu kham thảo:
* [Web Scraping with Python: Collecting Data from the Modern Web 1st Edition](https://www.amazon.com/Web-Scraping-Python-Collecting-Modern/dp/1491910291)
* [Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning 1st Edition](https://www.amazon.com/Applied-Text-Analysis-Python-Language-Aware/dp/1491963042)
* [Natural Language Processing with Python: Analyzing Text with the Natural Language Toolkit 1st Edition](https://www.amazon.com/Natural-Language-Processing-Python-Analyzing/dp/0596516495/ref=pd_sbs_2/138-2957899-5417957?pd_rd_w=bae9C&pf_rd_p=3676f086-9496-4fd7-8490-77cf7f43f846&pf_rd_r=HY0SV9DZKNKG3YP7XGCQ&pd_rd_r=8e2cac9c-9e76-47af-b450-2a0b45dab9ce&pd_rd_wg=aNUA6&pd_rd_i=0596516495&psc=1)
* ...